# MERRA2 SLV+Radiation Monthly Granularity Test
- Download
- Trim
- Create Cutout
- Generate Solar Outputs

## 1. Import and Download Data

In [1]:
import calendar
import geodata
import xarray as xr
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
DS_monthly = geodata.Dataset(module="merra2",
					 years=slice(1997, 1997),
					 months=slice(1,1),
                     weather_data_config = "slv_radiation_monthly")  

INFO:geodata.dataset:Directory /Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2 found, checking for completeness.
INFO:geodata.dataset:Directory complete.


In [2]:
if DS_monthly.prepared == False:
	DS_monthly.get_data()

## 2. Trim Variables

In [ ]:
## Variables before trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[0][1])
ds.data_vars

In [ ]:
DS_monthly.trim_variables()

In [4]:
## Variables after trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[0][1])
ds.data_vars

Data variables:
    albedo   (time, lat, lon) float32 ...
    swgdn    (time, lat, lon) float32 ...
    swtdn    (time, lat, lon) float32 ...
    t2m      (time, lat, lon) float32 ...

## 3. Create Cutout

In [3]:
monthly_cutout = geodata.Cutout(name="monthly-test-cutout",
                       module="merra2",
                       weather_data_config="slv_radiation_monthly",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(1997, 1997),
                       months=slice(1,1))

INFO:geodata.cutout:Cutout (monthly-test-cutout, /Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts) not found or incomplete.


In [4]:
monthly_cutout.prepare()

INFO:geodata.preparation:Starting preparation of cutout 'monthly-test-cutout'
INFO:geodata.datasets.merra2:MultiIndex([(1997, 1)],
           names=['year', 'month'])
INFO:geodata.datasets.merra2:[(1997, 1)]
INFO:geodata.preparation:1 tasks have been collected. Starting running them on all processors.
INFO:geodata.datasets.merra2:Opening `/Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2/1997/MERRA2_200.tavgM_2d_slv_rad_Nx.199701.nc4`
INFO:geodata.preparation:Merging variables into monthly compound files
INFO:geodata.preparation:Cutout 'monthly-test-cutout' has been successfully prepared


In [5]:
# Verify Cutout Contents
monthly_cutout.__dict__

{'name': 'monthly-test-cutout',
 'cutout_dir': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts/monthly-test-cutout',
 'prepared': True,
 'meta_append': 0,
 'config': 'slv_radiation_monthly',
 'dataset_module': <module 'geodata.datasets.merra2' from '/usr/local/lib/python3.7/site-packages/geodata-0.0.2-py3.7.egg/geodata/datasets/merra2.py'>,
 'meta': <xarray.Dataset>
 Dimensions:     (time: 1, x: 2, y: 3, year-month: 1)
 Coordinates:
   * time        (time) datetime64[ns] 1997-01-01T00:30:00
   * x           (x) float64 138.8 139.4
   * y           (y) float64 35.0 35.5 36.0
     lon         (x) float64 138.8 139.4
     lat         (y) float64 35.0 35.5 36.0
   * year-month  (year-month) MultiIndex
   - year        (year-month) int64 1997
   - month       (year-month) int64 1
 Data variables:
     *empty*
 Attributes:
     module:   merra2}

## 4. Generate Solar Output

In [6]:
ds_pv = geodata.convert.pv(
    monthly_cutout, 
    panel="KANENA", 
    orientation = "latitude_optimal"
    )
ds_pv

<xarray.DataArray 'AC power' (y: 3, time: 1, x: 2)>
array([[[0.10150946, 0.1014931 ]],

       [[0.10501715, 0.1024548 ]],

       [[0.10163419, 0.10006862]]])
Coordinates:
    lon      (x) float64 138.8 139.4
    lat      (y) float64 35.0 35.5 36.0
  * x        (x) float64 138.8 139.4
  * y        (y) float64 35.0 35.5 36.0
  * time     (time) datetime64[ns] 1997-01-01T00:30:00

In [11]:
df_pv = ds_pv.to_dataframe(name='pv')
df_pv

lon   lat        pv
y    time                x                               
35.0 1997-01-01 00:30:00 138.750  138.750  35.0  0.289056
                         139.375  139.375  35.0  0.280895
     1997-01-01 01:30:00 138.750  138.750  35.0  0.435743
                         139.375  139.375  35.0  0.422196
     1997-01-01 02:30:00 138.750  138.750  35.0  0.538005
...                                   ...   ...       ...
36.0 1997-01-31 21:30:00 139.375  139.375  36.0  0.000000
     1997-01-31 22:30:00 138.750  138.750  36.0  0.046439
                         139.375  139.375  36.0  0.049828
     1997-01-31 23:30:00 138.750  138.750  36.0  0.170946
                         139.375  139.375  36.0  0.172616

[4464 rows x 3 columns]

In [12]:
df_pv.to_csv('pv_test_monthly.csv')